In [6]:
import pandas as pd
books = pd.read_csv("../data/raw/Books.csv", delimiter=",")
#ratings = pd.read_csv("data/raw/Ratings.csv", delimiter=",")
#users = pd.read_csv("data/raw/Users.csv", delimiter=",")


#books['Year-Of-Publication']=int(books['Year-Of-Publication'])
#print(books.info())

books.head()

,Unnamed: 0,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'], errors='coerce')

In [11]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271357 entries, 0 to 271356
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Unnamed: 0           271357 non-null  int64 
 1   ISBN                 271357 non-null  object
 2   Book-Title           271357 non-null  object
 3   Book-Author          271355 non-null  object
 4   Year-Of-Publication  271357 non-null  int64 
 5   Publisher            271355 non-null  object
 6   Image-URL-S          271357 non-null  object
 7   Image-URL-M          271357 non-null  object
 8   Image-URL-L          271357 non-null  object
dtypes: int64(2), object(7)
memory usage: 18.6+ MB


In [14]:
# Sort by year (ascending)
books_sorted = books.sort_values(by='Year-Of-Publication', ascending=False)

# If you want the newest books first, use ascending=False
# books_sorted = books.sort_values(by='Year-Of-Publication', ascending=False)

# Show the result
print(books_sorted[['Book-Title', 'Year-Of-Publication']].head(10))

                                               Book-Title  Year-Of-Publication
80264   Alice's Adventures in Wonderland and Through t...                 2050
97826       Outline of European Architecture (Pelican S.)                 2050
116053                       Three Plays of Eugene Oneill                 2038
255406                               To Have and Have Not                 2037
260971        FOOTBALL SUPER TEAMS : FOOTBALL SUPER TEAMS                 2030
37487   MY TEACHER FRIED MY BRAINS (RACK SIZE) (MY TEA...                 2030
240166            In Our Time: Stories (Scribner Classic)                 2030
78168                                    Crossing America                 2030
228170       FOREST PEOPLE (Touchstone Books (Hardcover))                 2030
55676   MY TEACHER FLUNKED THE PLANET (RACK SIZE) (MY ...                 2030


In [12]:
print("books:")
print(books.head())
if books is None:
    print("Books data not loaded")
if "Year-Of-Publication" not in books.columns:
    print("Year-Of-Publication column not found")
books_clean = books.copy()
print("books_clean:")
print(books_clean.head())
if books_clean.empty:
    print("No valid year data found")
min_year = int(books_clean["Year-Of-Publication"].min())
max_year = int(books_clean["Year-Of-Publication"].max())
print("min_year:")
print(min_year)
print("max_year:")
print(max_year)

books:
   Unnamed: 0        ISBN                                         Book-Title  \
0           0  0195153448                                Classical Mythology   
1           1  0002005018                                       Clara Callan   
2           2  0060973129                               Decision in Normandy   
3           3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4           4  0393045218                             The Mummies of Urumchi   

            Book-Author  Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                 2002     Oxford University Press   
1  Richard Bruce Wright                 2001       HarperFlamingo Canada   
2          Carlo D'Este                 1991             HarperPerennial   
3      Gina Bari Kolata                 1999        Farrar Straus Giroux   
4       E. J. W. Barber                 1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \

In [17]:
import os
import requests
from dotenv import load_dotenv

# Load token dari .env file
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

# Cek apakah token tersedia
assert HF_TOKEN, "Token Hugging Face tidak ditemukan. Pastikan ada di .env dengan nama HF_TOKEN"

books = pd.read_csv("../data/processed/books_cleaned.csv")
book_embeddings = np.load("../models/saved/book_embeddings.npy")
model = SentenceTransformer('all-MiniLM-L6-v2')

query = "buku tentang psikologi"
query_vec = model.encode([query])
scores = cosine_similarity(query_vec, book_embeddings)[0]
top_idx = scores.argsort()[::-1][:5]
top_books = books.iloc[top_idx]

for _, row in top_books.iterrows():
    print(f"- {row['Book-Title']} by {row['Book-Author']} ({row['Year-Of-Publication']})")

def generate_huggingface_response(prompt, model="mistralai/Mistral-7B-Instruct-v0.1"):
    api_url = f"https://api-inference.huggingface.co/models/{model}"
    headers = {
        "Authorization": f"Bearer {HF_TOKEN}"
    }
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 200,
            "temperature": 0.7,
            "return_full_text": False
        }
    }

    response = requests.post(api_url, headers=headers, json=payload)

    if response.status_code == 200:
        output = response.json()
        return output[0]["generated_text"]
    else:
        print("❌ Hugging Face API Error:", response.status_code)
        print(response.text)
        return f"[Error {response.status_code}] {response.text}"


query = "recommend books about time travel"

retrieved_context = """
- The Time Machine by H.G. Wells (1895)
- 11/22/63 by Stephen King (2011)
- Kindred by Octavia E. Butler (1979)
- The First Fifteen Lives of Harry August by Claire North (2014)
- Outlander by Diana Gabaldon (1991)
"""

prompt = f"""prompt = "Suggest a book for someone who loves philosophy and time travel.
User asks: "{query}"

Based on the following relevant books:
{retrieved_context}

Please recommend the most suitable book(s) and explain your reasoning.
""".strip()

response = generate_huggingface_response(prompt)
print("📚 Hugging Face Response:\n")
print(response)


    


- Bukan legasi lalang by Sri Rahayu Mohd. Yusop (2001)
- Hatiku terluka di Bukhara by Hashim Haji Wan Teh (2002)
- Biko by Donald Woods (1991)
- Biko by Donald Woods (1987)
- ABIKU by Debo Kotun (1999)
❌ Hugging Face API Error: 404
Not Found
📚 Hugging Face Response:

[Error 404] Not Found


In [30]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load token dari .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Konfigurasi Google GenAI
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

def generate_gemini_response(prompt):
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"❌ Gemini API Error: {e}")
        return f"[Error] {e}"

# Contoh penggunaan
query = "recommend books about time travel"
retrieved_context = """
- The Time Machine by H.G. Wells (1895)
- 11/22/63 by Stephen King (2011)
- Kindred by Octavia E. Butler (1979)
"""

prompt = f"""
User asks: "{query}"

Here are some related books:
{retrieved_context}

Which of these would you recommend and why?
"""

response = generate_gemini_response(prompt)
print("📚 Gemini Response:\n")
print(response)

📚 Gemini Response:

Okay, based on the prompt and the provided list, here's my recommendation and why:

**I would recommend all three, but with slightly different caveats and focusing on what makes them great:**

*   **The Time Machine by H.G. Wells (1895):** This is a foundational text in the time travel genre and should be read by anyone interested in the topic.
    *   **Why:** It's a classic. It establishes many of the core concepts of time travel literature, including the technological means of travel and the exploration of future societies. While the writing style is a bit dated, it's a relatively short read and provides a powerful social commentary on Victorian England.
    *   **Caveat:** It is very old. The language and social commentary reflect the time it was written. Some modern readers may find it slow-paced or the social commentary heavy-handed.

*   **11/22/63 by Stephen King (2011):** This is a great choice if you enjoy well-developed characters, historical fiction, and